In [1]:
!pip install anthropic gradio bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [2]:
import openai
from openai import OpenAI
import anthropic
import os
import gradio as gr
from google.colab import userdata
from huggingface_hub import login
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, BitsAndBytesConfig
import threading
from queue import Queue


In [3]:
hf_token = userdata.get('HF_TOKEN')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GROK_API_KEY = userdata.get('GROK_API_KEY')
CLAUDE_API_KEY = userdata.get('ANTHROPIC_API_KEY')


In [4]:
if not hf_token:
    print("Warning: HF_TOKEN not found")
if not OPENAI_API_KEY:
    print("Warning: OPENAI_API_KEY not found")
if not GROK_API_KEY:
    print("Warning: GROK_API_KEY not found")
if not CLAUDE_API_KEY:
    print("Warning: ANTHROPIC_API_KEY not found")

login(hf_token, add_to_git_credential=True)

In [5]:
openai_client = OpenAI(api_key=OPENAI_API_KEY)
claude_client = anthropic.Anthropic(api_key=CLAUDE_API_KEY)
grok_client = OpenAI(api_key=GROK_API_KEY, base_url="https://api.x.ai/v1")


In [24]:
## Closed Source Models
OPENAI_MODEL = 'gpt-4o-mini'
CLAUDE_MODEL = 'claude-3-7-sonnet-20250219'
GROK_MODEL = 'grok-2-1212'

## Open Source Models
CODE_LAMA = 'codellama/CodeLlama-13b-Instruct-hf'

In [32]:
system_message = """You are a specialized code conversion assistant with expertise in multiple programming languages.
  Your primary function is to convert code between languages accurately while preserving functionality and idiomatic practices
  of the target language.

  Core Responsibilities:
  - Convert source code from C++, JavaScript, Python, or PHP into any of these target languages: PHP, Python, JavaScript, or Golang
  - Maintain the original code's logic, functionality, and intent in the converted code
  - Add clear and appropriate comments and docstrings in the target language's conventional style.
  - Write comprehensive unit tests when requested
  - Explain key differences between the source and target languages when relevant

  Conversion Guidelines:
  1. Preserve Functionality: Ensure the converted code performs the same operations as the original code
  2. Follow Target Language Conventions: Use idiomatic patterns and best practices for the target language
  3. Maintain Readability: Write clean, well-structured code with proper indentation and spacing
  4. Handle Language-Specific Features: Address differences in language features such as:
    - Type systems (static vs. dynamic typing)
    - Memory management (manual vs. garbage collection)
    - Concurrency models
    - Error handling approaches
    - Standard libraries and ecosystem

  Documentation Standards:
  - Add clear and comprehensive docstrings/comments following the target language's conventions:
    - PHP: PHPDoc style comments
    - Python: PEP 257 docstrings
    - JavaScript: JSDoc style comments
    - Golang: Go standard comment format

  Unit Testing Standards
  When asked to create unit tests:
  - Write tests covering the main functionality of the code
  - Include tests for edge cases and potential error conditions
  - Use the appropriate testing framework for the target language:
    - PHP: PHPUnit
    - Python: unittest or pytest
    - JavaScript: Jest or Mocha
    - Golang: Go's built-in testing package

  Response Format
  1. Provide the converted code in an appropriately formatted code block
  2. When significant implementation choices were made, briefly explain your choices
  3. If requested, provide unit tests in a separate code block
  4. When helpful, note important differences between languages that might affect usage

  Development Environment Considerations
  - For language-specific package or dependency management, include brief instructions on how to set up the environment to run the code
  - Mention any required libraries or frameworks needed to execute the converted codes """

In [ ]:
print(system_message)

You are a specialized code conversion assistant with expertise in multiple programming languages.
  Your primary function is to convert code between languages accurately while preserving functionality and idiomatic practices
  of the target language.

  Core Responsibilities:
  - Convert source code from C++, JavaScript, Python, or PHP into any of these target languages: PHP, Python, JavaScript, or Golang
  - Maintain the original code's logic, functionality, and intent in the converted code
  - Add appropriate comments/docstrings in the target language's conventional style
  - Write comprehensive unit tests when requested
  - Explain key differences between the source and target languages when relevant

  Conversion Guidelines:
  1. Preserve Functionality: Ensure the converted code performs the same operations as the original code
  2. Follow Target Language Conventions: Use idiomatic patterns and best practices for the target language
  3. Maintain Readability: Write clean, well-stru

In [34]:
def get_user_prompt(from_lang, to_lang, code, docstring=False, unit_test=False):

    if docstring:
        user_prompt = f"Write clear comments and docstrings for the {from_lang} code. Here is the code \nf{code}"

        return user_prompt

    elif unit_test:
         user_prompt = f"Write a comprehensive unit test for the {from_lang} code. Include necessary packages as well. Here is the code \nf{code}"

         return user_prompt
    else:
         user_prompt = f"Convert the following {from_lang} code into {to_lang}. Maintain the structure and intent of the code."

         user_prompt += f"\n\nHere's the {from_lang} code to convert:\n\n```{from_lang.lower()}\n{code}\n```"

         return user_prompt


In [9]:
def get_quantization_config():
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    return quant_config

In [10]:
def get_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_auth_token=True,
        trust_remote_code=True
    )
    return tokenizer

In [11]:
def get_model(model_name):
    quant_config = get_quantization_config()
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map='auto',
        torch_dtype=torch.float16,
        use_auth_token=True,
        quantization_config=quant_config,
        trust_remote_code=True
    )
    return model

In [12]:
def get_messages(from_lang, to_lang, code, docstring=False, unit_test=False):
    messages = [
        {
            'role': 'system',
            'content': system_message
        },
        {
            'role': 'user',
            'content': get_user_prompt(from_lang, to_lang, code, docstring, unit_test)
        }
    ]
    return messages


In [35]:
def stream_closed_source_model(model_type, model_name, from_lang, to_lang, code, docstring=False, unit_test=False):
    messages = get_messages(from_lang, to_lang, code, docstring, unit_test)
    accumulated_text = ""

    try:
        if model_type == "openai":
            response = openai_client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": m["role"], "content": m["content"]}
                    for m in messages
                ],
                temperature=0.7,
                max_tokens=4096,
                stream=True
            )

            for chunk in response:
                if chunk.choices[0].delta.content is not None:
                    accumulated_text += chunk.choices[0].delta.content
                    yield accumulated_text

        elif model_type == "anthropic":
            with claude_client.messages.stream(
                model=model_name,
                system=system_message,
                messages=[
                   messages[1]
                ],
                max_tokens=4096
            ) as stream:
                for text in stream.text_stream:
                    accumulated_text += text
                    yield accumulated_text

        elif model_type == "grok":
            response = grok_client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": m["role"], "content": m["content"]}
                    for m in messages
                ],
                temperature=0.7,
                max_tokens=4096,
                stream=True
            )

            for chunk in response:
                if chunk.choices[0].delta.content is not None:
                    accumulated_text += chunk.choices[0].delta.content
                    yield accumulated_text

    except Exception as e:
        error_msg = f"API Error: {str(e)}"
        accumulated_text += f"\n{error_msg}"
        yield accumulated_text

In [36]:
def stream_open_source_model(model_name, from_lang, to_lang, code, docstring=False, unit_test=False):
    try:
        tokenizer = get_tokenizer(model_name)
        model = get_model(model_name)

        messages = get_messages(from_lang, to_lang, code, docstring, unit_test)
        text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

        model_inputs = tokenizer(text, return_tensors='pt').to('cuda')


        streamer = TextIteratorStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


        queue = Queue()


        def generate():
            try:
                with torch.no_grad():
                    model.generate(
                        **model_inputs,
                        max_new_tokens=4096,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.95,
                        streamer=streamer
                    )
            except Exception as e:
                queue.put(f"Error in generation: {str(e)}")


        thread = threading.Thread(target=generate)
        thread.start()


        accumulated_text = ""


        for text_chunk in streamer:
            accumulated_text += text_chunk
            yield accumulated_text

        thread.join()


        if not queue.empty():
            error_msg = queue.get()
            accumulated_text += f"\n{error_msg}"
            yield accumulated_text

    except Exception as e:
        yield f"Error initializing model: {str(e)}"

In [25]:
def convert_code(model_choice, from_lang, to_lang, code, include_docstring, include_tests):
    # Validate input code
    if not code or len(code.strip()) == 0:
        yield "Please enter some source code to convert"
        return

    if model_choice == "OpenAI GPT-4o Mini":
        yield from stream_closed_source_model("openai", OPENAI_MODEL, from_lang, to_lang, code, include_docstring, include_tests)
    elif model_choice == "Anthropic Claude 3.7 Sonnet":
        yield from stream_closed_source_model("anthropic", CLAUDE_MODEL, from_lang, to_lang, code, include_docstring, include_tests)
    elif model_choice == "Grok 2.1":
        yield from stream_closed_source_model("grok", GROK_MODEL, from_lang, to_lang, code, include_docstring, include_tests)
    elif model_choice == "Code Lamma 13B":
        yield from stream_open_source_model(CODE_LAMA, from_lang, to_lang, code, include_docstring, include_tests)
    else:
        yield "Please select a valid model"

In [40]:
def create_gradio_interface():
    with gr.Blocks(title="Code Language Converter") as app:
        gr.Markdown("# Code Language Converter")
        gr.Markdown("Convert code between different programming languages using AI models")

        with gr.Row():
            with gr.Column():
                model_choice = gr.Dropdown(
                    choices=[
                        "OpenAI GPT-4o Mini",
                        "Anthropic Claude 3.7 Sonnet",
                        "Grok 2.1",
                        "Code Lamma 13B",
                    ],
                    label="Select AI Model",
                    value="Anthropic Claude 3.7 Sonnet"
                )

                from_lang = gr.Dropdown(
                    choices=["Python", "JavaScript", "PHP", "C++", "Golang"],
                    label="Source Language",
                    value="Python"
                )

                to_lang = gr.Dropdown(
                    choices=["Python", "JavaScript", "PHP", "Golang"],
                    label="Target Language",
                    value="JavaScript"
                )

                with gr.Row():
                    include_docstring = gr.Checkbox(label="Write Docstrings", value=True)
                    include_tests = gr.Checkbox(label="Write Unit Tests", value=False)

                convert_button = gr.Button("Convert Code", variant="primary")

        with gr.Row():
            with gr.Column():
                source_code = gr.TextArea(
                    label="Source Code",
                    placeholder="Paste your code here...",
                    lines=15
                )

            with gr.Column():
                output = gr.Markdown(
                    label="Converted Code",
                )

        convert_button.click(
            fn=convert_code,
            inputs=[model_choice, from_lang, to_lang, source_code, include_docstring, include_tests],
            outputs=output
        )

        gr.Markdown("""
        ### How to use:
        1. Select your AI model of choice
        2. Choose the source and target programming languages
        3. Optionally select to include docstrings and/or unit tests
        4. Paste your code in the Source Code box
        5. Click "Convert Code" button and watch the response stream in real-time


        """)

    return app


In [41]:
# Launch the Gradio interface
if __name__ == "__main__":
    app = create_gradio_interface()
    app.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1348d41ac31c805446.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1348d41ac31c805446.gradio.live
